In [ ]:
import sys
print(sys.executable)

c:\Users\Linh\Documents\financial_news_sentiment\.venv\Scripts\python.exe


In [45]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import spacy
from spacy_sentiws import spaCySentiWS
from datasets import load_dataset

In [ ]:
# Auf Finanznachrichten vortrainiertes Modell
# Basiert auf GermanFinBert und wurde für Sentimentanalyse im Finanzbereich optimiert
# Weitere Informationen unter: https://huggingface.co/scherrmann/GermanFinBert_SC_Sentiment
model_name = "scherrmann/GermanFinBert_SC_Sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Vortrainiertes Modell laden für Sentimentanalyse
fin_model = AutoModelForSequenceClassification.from_pretrained(model_name)
# Aufbau Pipeline, um Tokenizer und Modell zu kombinieren
fin_pipeline = pipeline("sentiment-analysis", model=fin_model, tokenizer=tokenizer)

sentence_model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1') # Multilinguales paraphrase-xlm-r-multilingual-v1 Modell zur Berechnung von Satz-/ Dokument-Embeddings


In [18]:
# Pfad zur Excel-Datei mit manuell klassifizierten Sentiment-Daten
file_path = './sentiment_data/manual_sentiment.xlsx'
data = pd.read_excel(file_path)

In [39]:
# lexikalischer Ansatz
nlp = spacy.load('de_core_news_sm')
nlp.add_pipe('sentiws', config={'sentiws_path': './sentiment_lexicon/SentiWS_v2.0'})


In [40]:
# Laden des BPW-Dictionarys (spezielles Finanzlexikon)
bpw_path = './sentiment_lexicon/BPW_Dictionary.xlsx'
bpw_data = pd.read_excel(bpw_path, sheet_name=None)  # Alle Blätter laden

# Blätter der Excel-Datei in das jeweilige Sentiment einordnen mit Begriffen als dictionary
bpw_lexicon = {
    "positiv": pd.read_excel(bpw_path, sheet_name="POS_BPW").dropna().values.flatten().tolist(),
    "negativ": pd.read_excel(bpw_path, sheet_name="NEG_BPW").dropna().values.flatten().tolist(),
    "neutral": pd.read_excel(bpw_path, sheet_name="UNC_BPW").dropna().values.flatten().tolist()
}

In [ ]:
# Klasse zur hybriden Sentiment-Analyse (Transformer, Lexika und Satz-Embeddings kombiniert)
class SentimentAnalysis:
  def __init__(self, sentence_model, tokenizer, pipeline, nlp, bpw_lexicon, config=None):
    self.sentence_model = sentence_model
    self.tokenizer = tokenizer
    self.pipeline = pipeline
    self.nlp = nlp
    self.bpw_lexicon = bpw_lexicon
    # Konfigurationsparameter
    self.config = config or {
        "bert_weight": 0.6, # Gewichtung des BERT-Scores
        "lexical_weight": 0.6, # Gewichtung des lexikalischen Scores 
        "threshold_pos": 0.1, # Schwellenwert für positive Sentiments
        "threshold_neg": -0.1,  # Schwellenwert für negative Sentiments
        "embedding_threshold": 0.2, # Schwellenwert für Satzähnlichkeit
        "max_tokens": 512,  # Maximale Tokenanzahl pro Fenster
        "overlap": 25 # Überlappung zwischen Fenstern
    }
  # Satz Embeddings zur Relevanz-Bewertung
  def filter_sentences_by_embedding(self, text, reference_sentences): # Inspiration von https://huggingface.co/tasks/sentence-similarity und https://ubiai.tools/from-words-to-vectors-a-dive-into-spacy-transformers-for-embeddings/
    doc = nlp(text)
    # Zerlegung des Text in Sätze mit Spacy
    sentences = [sent.text for sent in doc.sents]
    # Kodierung der Sätze in Vektorepräsentationen unter Verwendung des Sentence Transformer-Modells
    sentence_embeddings = self.sentence_model.encode(sentences, convert_to_tensor=True)
    # Referenzsätze aus der Phrasebank für spätere Berechnung
    reference_embeddings = self.sentence_model.encode(reference_sentences, convert_to_tensor=True)
    # Berechne die Ähnlichkeit aller Sätze mit den Referenz-Embeddings
    similarities = cosine_similarity(sentence_embeddings, reference_embeddings)
    # Sätze mit höchster Ähnlichkeit filtern
    filtered_sentences = [sentence for sentence, sim in zip(sentences, similarities) if max(sim) >= self.config["embedding_threshold"]]

    # Rückgabe der gefilterten Sätze als Text
    return " ".join(filtered_sentences)

  # Aufteilung der Texte aufgrund des Token-limits
  def create_rolling_windows(self, text):
    # Text Tokenisierung
    tokens = self.tokenizer(text, truncation=True, max_length=self.config["max_tokens"])["input_ids"]
    windows = []
    start = 0
    # Dekodieren der Tokens zu Text und Hinzufügen zur Fensterliste
    while start < len(tokens):
        end = min(start + self.config["max_tokens"], len(tokens))
        window_tokens = tokens[start:end]
        windows.append(self.tokenizer.decode(window_tokens, skip_special_tokens=True))
        # Überlappung der Fenster
        start += self.config["max_tokens"] - self.config["overlap"]

    return windows

  # labels --> numerische Scores
  def get_bert_score(self, label):
    label_mapping = {"Positiv": 1, "Negativ": -1, "Neutral": 0}
    # label_mapping = {"positive": 1, "negative": -1, "neutral": 0}
    # Falls Label unbekannt, 0 zurückgegeben
    return label_mapping.get(label, 0)

  def calculate_lexical_score(self, text):
    doc = self.nlp(text)
    # Summiere die SentiWS-Scores aller Tokens im Text
    sentiws_score = sum(token._.sentiws for token in doc if token._.sentiws is not None) # basierend auf https://spacy.io/universe/project/spacy-sentiws
    # Summiere die BPW-Scores basierend auf positiven und negativen Wörtern
    bpw_score = sum(
        1 if word in self.bpw_lexicon["positiv"] else -1 if word in self.bpw_lexicon["negativ"] else 0
        for word in text.lower().split()
    )
    # Kombinieren der Scores anhand der Gewichtung in der config
    return (
        self.config["lexical_weight"] * sentiws_score
        + (1 - self.config["lexical_weight"]) * bpw_score
    )

  def analyze_text(self, text, reference_sentences):
    filtered_text = self.filter_sentences_by_embedding(text, reference_sentences)
    # Erstellung von Fenstern bei langen Texten
    windows = self.create_rolling_windows(filtered_text)

    results = []
    total_bert_score, total_combined_score = 0, 0

    for window in windows:
        # Berechne Sentiment-Score mit der BERT-Pipeline
        bert_result = self.pipeline(window)[0]
        bert_score = self.get_bert_score(bert_result["label"])
        # Berechne lexikalischen Sentiment-Score
        lexical_score = self.calculate_lexical_score(window)

        # Kombinierte Bewertung basierend auf konfigurierter Gewichtung
        combined_score = (
            self.config["bert_weight"] * bert_score +
            (1 - self.config["bert_weight"]) * lexical_score
        )

        total_bert_score += bert_score
        total_combined_score += combined_score

        results.append({"text": window, "bert_score": bert_score, "combined_score": combined_score})

    # Berechne Durchschnittswerte und gewichtete Ergebnisse
    avg_bert_score = total_bert_score / len(windows)
    avg_combined_score = total_combined_score / len(windows)
    
    return {
        "avg_bert_score": avg_bert_score,
        "avg_combined_score": avg_combined_score,
        "segment_results": results
    }

  @staticmethod
  def score_to_label(score, threshold_pos=0.1, threshold_neg=-0.1):
    if score > threshold_pos:
        return "positiv"
    elif score < threshold_neg:
        return "negativ"
    else:
        return "neutral"
    
    # feinere sentimentklassen 
    #   def score_to_label(score, label_thresholds):
    # if score >= label_thresholds["strong_positive"]:
    #     return "stark positiv"
    # elif score >= label_thresholds["weak_positive"]:
    #     return "schwach positiv"
    # elif score > label_thresholds["weak_negative"]:
    #     return "neutral"
    # elif score >= label_thresholds["strong_negative"]:
    #     return "schwach negativ"
    # else:
    #     return "stark negativ"
    
  # Bewertung der Ergebnisse
  def evaluate(self, data, reference_sentences, output_file=None):
    correct_predictions = 0
    y_true, y_pred = [], [] # Listen für tatsächliche und vorhergesagte Labels
    results = []

    for _, row in data.iterrows():
        text, expected_sentiment = row["Text"], row["Sentiment"]
        analysis_result = self.analyze_text(text, reference_sentences)

        # Durchschnittsscore berechnen und in ein Label umwandeln
        score = analysis_result["avg_combined_score"]
        predicted_label = self.score_to_label(score)

        y_true.append(expected_sentiment)
        y_pred.append(predicted_label)

        if predicted_label == expected_sentiment:
            correct_predictions += 1

        results.append({
            "Expected Sentiment": expected_sentiment,
            "Predicted Sentiment": predicted_label,
            "Average BERT Score": analysis_result["avg_bert_score"],
            "Average Combined Score": analysis_result["avg_combined_score"],
        })

    accuracy = correct_predictions / len(data)

    # Berechnung der Performance-Metriken
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    # Berechnung der Confusion Matrix
    cm = confusion_matrix(y_true, y_pred, labels=["positiv", "neutral", "negativ"])

    if output_file:
        data["Predicted Sentiment"] = y_pred
        data.to_excel(output_file, index=False)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": cm,
        "results": results
    }

In [56]:
# hohes embedding --> niedrige accuracy
# Load reference sentences
# 0.3 = 80%
# Load Financial PhraseBank
dataset = load_dataset("scherrmann/financial_phrasebank_75agree_german")

# Extract reference sentences
positive_reference_sentences = [row['sentence'] for row in dataset['train'] if row['label'] == 2][:10]
negative_reference_sentences = [row['sentence'] for row in dataset['train'] if row['label'] == 0][:10]

# Combine positive and negative references
reference_sentences = positive_reference_sentences + negative_reference_sentences

# Sentiment Analysis
analyzer = SentimentAnalysis(sentence_model, tokenizer, fin_pipeline, nlp, bpw_lexicon)

evaluation_results = analyzer.evaluate(data, reference_sentences, output_file="./sentiment_data/sentiment_results.xlsx")

In [57]:
# Ausgabe aller Metriken
# embedding threshold 0.2
print(f"Accuracy: {evaluation_results['accuracy']:.2f}")
print(f"Precision: {evaluation_results['precision']:.2f}")
print(f"Recall: {evaluation_results['recall']:.2f}")
print(f"F1-Score: {evaluation_results['f1_score']:.2f}")

print("\nConfusion Matrix:")
print(evaluation_results["confusion_matrix"])

print("\nDetailed Results:")
for result in evaluation_results["results"][:5]:  # Begrenzung auf die ersten 5 Ergebnisse
    print(result)

Accuracy: 0.81
Precision: 0.81
Recall: 0.81
F1-Score: 0.80

Confusion Matrix:
[[179  10   9]
 [ 18  18  13]
 [  3   1  30]]

Detailed Results:
{'Expected Sentiment': 'neutral', 'Predicted Sentiment': 'positiv', 'Average BERT Score': 0.0, 'Average Combined Score': 0.29322400000000004}
{'Expected Sentiment': 'positiv', 'Predicted Sentiment': 'positiv', 'Average BERT Score': 1.0, 'Average Combined Score': 0.27292799999999995}
{'Expected Sentiment': 'positiv', 'Predicted Sentiment': 'positiv', 'Average BERT Score': 1.0, 'Average Combined Score': 0.821424}
{'Expected Sentiment': 'negativ', 'Predicted Sentiment': 'negativ', 'Average BERT Score': -1.0, 'Average Combined Score': -0.76}
{'Expected Sentiment': 'negativ', 'Predicted Sentiment': 'negativ', 'Average BERT Score': -1.0, 'Average Combined Score': -0.621792}
